# ETL

Aquí se haran las transformaciones necesarias para tener datasets para optimizar las consultas

In [1]:
import pandas as pd
import numpy as np


In [2]:
df_games=pd.read_parquet(r'Datasets/steam_games_clean.parquet')

In [3]:
df_users=pd.read_parquet(r'Datasets/users_items_clean.parquet')

In [4]:
df_reviews=pd.read_parquet(r'Datasets/user_reviews_clean.parquet')

In [5]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865427 entries, 0 to 3865426
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int16  
 2   item_id           int32  
 3   playtime_forever  float16
dtypes: float16(1), int16(1), int32(1), object(1)
memory usage: 59.0+ MB


In [6]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27427 entries, 0 to 27426
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27427 non-null  object 
 1   app_name      27427 non-null  object 
 2   tags          27427 non-null  object 
 3   price         27427 non-null  float64
 4   id            27427 non-null  int32  
 5   developer     27427 non-null  object 
 6   release_year  27427 non-null  object 
dtypes: float64(1), int32(1), object(5)
memory usage: 1.4+ MB


In [7]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  bool  
 3   review     59305 non-null  object
dtypes: bool(1), int32(1), object(2)
memory usage: 1.2+ MB


## Análisis de sentimiento
En esta sección antes de realizar otras transformaciones se realizara el análisis de sentimientos mediante la librería **TextBlob** para simplificar el proceso. Pero esta análisis se puede realizar con otras librerías más complejas o mas elaboradas.

In [8]:
from textblob import TextBlob

#Se crea una función para devolver el sentimiento de un review pero en un rango de 0 a 2
def sentiment(review):
    blob=TextBlob(review)
    resultado=1 + round(blob.sentiment.polarity)
    return resultado

In [9]:
df_reviews['sentiment']=df_reviews['review'].apply(sentiment)

In [10]:
df_reviews.drop(columns=['review'],inplace=True)

In [11]:
df_reviews

,user_id,item_id,recommend,sentiment
0,76561197970982479,1250,True,1
1,76561197970982479,22200,True,1
2,76561197970982479,43110,True,1
3,js41637,251610,True,1
4,js41637,227300,True,1
...,...,...,...,...
59300,76561198312638244,70,True,1
59301,76561198312638244,362890,True,1
59302,LydiaMorley,273110,True,1
59303,LydiaMorley,730,True,2


## Transformación
Una vez finalizado el análisis de sentimiento vamos a comenzar a transformar y unir los datasets

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  bool  
 3   sentiment  59305 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(1)
memory usage: 1.2+ MB


In [13]:
df_item_sentiment_counts = df_reviews.groupby('item_id')['sentiment'].value_counts().unstack().reset_index()

In [14]:
df_item_sentiment_counts

sentiment,item_id,0,1,2
0,10,1.0,49.0,7.0
1,20,NaN,16.0,1.0
2,30,NaN,3.0,1.0
3,40,NaN,1.0,NaN
4,50,NaN,4.0,NaN
...,...,...,...,...
3677,521340,NaN,NaN,2.0
3678,521430,NaN,1.0,NaN
3679,521570,1.0,1.0,NaN
3680,521990,NaN,1.0,NaN


In [15]:
df_item_recommend_count=df_reviews.groupby('item_id')['recommend'].value_counts().unstack().reset_index()


In [16]:
df_counts=pd.merge(df_item_sentiment_counts,df_item_recommend_count,on='item_id')

In [17]:
df_counts.fillna(0,inplace=True)

In [18]:
df_developers=pd.merge(df_games[['price','developer','release_year','id']],df_counts,left_on='id',right_on='item_id')

In [19]:
df_developers.drop('id',axis=1,inplace=True)

In [20]:
df_developers.rename(columns={'0_x':'Negative','1_x':'Neutral',2:'Positive','False_y':'False','True_y':'True'},inplace=True)

In [21]:
df_developers = df_developers.astype({'Negative': 'int16', 'Neutral': 'int16', 'Positive': 'int16', 'False': 'int16', 'True': 'int16'})

In [22]:
df_developers.to_parquet(r'Datasets/developers.parquet')

In [29]:
df_developers[df_developers['developer']=='Valve']

,price,developer,release_year,item_id,Negative,Neutral,Positive,False,True


In [24]:
df_developers['developer'].unique()

array(['Stainless Games Ltd', 'Valve', 'Outerlight Ltd.', ...,
       'Neversoft', 'Malfador Machinations', 'Strategy First'],
      dtype=object)

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [25]:
desarrollador="valve"
desarrollador=desarrollador.title()
desarrollador

'Valve'

In [33]:
df_developers[df_developers['developer']==desarrollador][['Positive','Negative']].sum()

Positive    1009
Negative     113
dtype: int64